In [14]:
from diffrobot.robot.visualizer import RobotViz
import os
import json
import pdb
import time
import spatialmath as sm
import numpy as np
import cv2
import open3d as o3d
import matplotlib.pyplot as plt
import pyrealsense2 as rs


dataset_path = "/home/krishan/work/2024/datasets/image_processing"
episodes = sorted(os.listdir(os.path.join(dataset_path, "episodes")), key=lambda x: int(x))
# env = RobotViz()


# read in camera intrinsics
intrinsics_path = os.path.join(dataset_path, "calibration", "hand_eye.json")
with open(intrinsics_path, "r") as f:
    intrinsics_data = json.load(f)


intrinsics_matrix = np.array(intrinsics_data["intrinsics"])

fx, fy, cx, cy = intrinsics_matrix[0, 0], intrinsics_matrix[1, 1], intrinsics_matrix[0, 2], intrinsics_matrix[1, 2]

X_FE = np.array([[0.70710678, 0.70710678, 0.0, 0.0], 
                [-0.70710678, 0.70710678, 0, 0], 
                [0.0, 0.0, 1.0, 0.1], 
                [0.0, 0.0, 0.0, 1.0]])

X_FE = sm.SE3(X_FE, check=False).norm()


def to_3D(fx, fy, depth, cx, cy, u, v):
    x = (u-cx)*depth/fx
    y = (v-cy)*depth/fy
    z = depth
    x = np.expand_dims(x, axis = -1)
    y = np.expand_dims(y, axis = -1)
    z = np.expand_dims(z, axis = -1)
    return np.concatenate((x,y,z), axis=-1)
    



In [21]:

episode_path = os.path.join(dataset_path, "episodes", "0", "state.json")
with open(episode_path, "r") as f:
    data = json.load(f)

# for idx, state in enumerate(data):

# rgb_fpath = os.path.join(dataset_path, "episodes", episode, "images", "rgb", f"{idx+1}.png")
rgb_fpath = os.path.join(dataset_path, "episodes", "0", "images", "rgb", "1.png")
# depth_fpath = os.path.join(dataset_path, "episodes", episode, "images", "depth", f"{idx+1}.png")
depth_fpath = os.path.join(dataset_path, "episodes", "0", "images", "depth", "1.png")
rgb = cv2.imread(rgb_fpath)
depth = cv2.imread(depth_fpath, -1) * 0.00025
# create a mask that cuts out a border of the image
mask = np.zeros_like(depth)
mask[50:-50, 50:-50] = 1
depth = depth * mask
rgb = rgb * mask[..., np.newaxis].astype(np.uint8)

# mask 2
mask = depth< 1.0
depth = depth * mask
rgb = rgb * mask[..., np.newaxis].astype(np.uint8)

height, width, _ = rgb.shape
intrinsics = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)

H = rgb.shape[0]
W = rgb.shape[1]
u = np.arange(W)
v = np.arange(H)
u, v = np.meshgrid(u, v)
xyz = to_3D(fx, fy, depth, cx, cy, u, v)
point_cloud = np.concatenate((xyz, rgb), axis=-1)


# Separate XYZ coordinates and RGB colors
xyz = point_cloud[..., :3]  # Shape: (720, 1280, 3)
rgb = point_cloud[..., 3:6] / 255.0  # Normalizing RGB to [0, 1], Shape: (720, 1280, 3)

# Flatten the arrays since Open3D expects N x 3 matrices for points and colors
xyz_flat = xyz.reshape(-1, 3)
rgb_flat = rgb.reshape(-1, 3)

# Create a point cloud object
pcd = o3d.geometry.PointCloud()

# Set the points and colors
pcd.points = o3d.utility.Vector3dVector(xyz_flat)
pcd.colors = o3d.utility.Vector3dVector(rgb_flat)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])




# # # show image with pyplot
# plt.figure()
# # use a different colormap for depth
# # color bar is in meters
# # add a colorbar to the image
# plt.colorbar(plt.imshow(depth, cmap='jet'))
# plt.imshow(depth, cmap='jet')

# # show rgb image as  well
# plt.figure()
# plt.imshow(rgb)

# plt.show()


        


[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [17]:
point_cloud.shape

(720, 1280, 6)

In [20]:

rgb
    

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [13]:




        # 

        # Create an Open3D RGBD image from the RGB and depth images
        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
            o3d.geometry.Image(rgb),
            o3d.geometry.Image(depth),
            depth_scale=1.0/0.00025,  # Adjust based on your depth image format
            depth_trunc=2.0,  # Adjust to the maximum depth value you're interested in
            convert_rgb_to_intensity=False
        )
 
        # Create a point cloud from the RGBD image and the intrinsic parameters
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
            rgbd_image,
            intrinsics
        )

        
        # Visualize the point cloud
        o3d.visualization.draw_geometries([pcd])

        plt.show()

        # env.ee_pose.T = sm.SE3(state["X_BE"], check=False).norm()
        # target_pose = env.robot.fkine(state["gello_q"], "panda_link8") * X_FE
        # env.policy_pose.T = target_pose 
        # env.step(state["robot_q"])
        # time.sleep(0.1)

RuntimeError: Image can only be initialized from buffer of uint8, uint16, or float!